In [2]:
!git clone https://github.com/gefeiwang/scMODAL.git

Cloning into 'scMODAL'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 57 (delta 18), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (57/57), 1.18 MiB | 10.48 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [3]:
!cp -r scMODAL/scmodal scmodal

In [ ]:
!pip install anndata scanpy numba umap-learn torch annoy

In [ ]:
import anndata
import scanpy as sc
import numpy as np 
import pandas as pd 
from sklearn.utils.extmath import randomized_svd

In [8]:
import requests, zipfile, io, os
local_zip_path = "maxfuse_data.zip"

# Check if the zip file already exists locally
if not os.path.exists(local_zip_path):
    # Download the zip file
    r = requests.get("http://stat.wharton.upenn.edu/~zongming/maxfuse/data.zip")
    with open(local_zip_path, 'wb') as f:
        f.write(r.content)

# Load the zip file from local storage
with zipfile.ZipFile(local_zip_path, 'r') as z:
    z.extractall("../")


In [ ]:
import numpy as np
import pandas as pd
from scipy.io import mmread
import anndata as ad
import scanpy as sc
# import maxfuse as mf

In [ ]:
import numpy as np
from sklearn.utils.extmath import randomized_svd

def compute_correlated_pairs(rna_adata, protein_adata, k,  correspondence, output_csv_path='correlated_pairs.csv'):
    
    # Copy the input data
    rna_adata_cca = rna_adata.copy()
    protein_adata_cca = protein_adata.copy()

    # Group data by cell types and compute mean
    rna_grouped = rna_adata_cca.to_df().groupby(rna_adata.obs['celltype.l1']).mean()
    protein_grouped = protein_adata_cca.to_df().groupby(protein_adata.obs['celltype.l1']).mean()

    # Find common cell types and reshape data matrices
    common_index = rna_grouped.index.intersection(protein_grouped.index)
    rna_grouped = rna_grouped.loc[common_index]
    protein_grouped = protein_grouped.loc[common_index]
    print("Data grouped...")

    # Convert grouped dataframes to numpy arrays
    rna_cca_data = rna_grouped.values
    protein_cca_data = protein_grouped.values

    # Ensure both matrices have the same shape
    assert rna_cca_data.shape[0] == protein_cca_data.shape[0], "RNA and Protein data must have the same set of cells."

    # Log + 1 Normalization
    rna_cca_data = np.log1p(rna_cca_data)
    protein_cca_data = np.log1p(protein_cca_data)
    print("Data Normalized...")

    # Mean-centering the RNA and Protein data
    rna_centered = rna_cca_data - np.mean(rna_cca_data, axis=0)
    protein_centered = protein_cca_data - np.mean(protein_cca_data, axis=0)
    print("Data Centered...")

    # Computing the covariance matrix
    covariance_matrix = np.cov(rna_centered.T, protein_centered.T)
    print("Covariance Matrix Computed...")
    print(covariance_matrix.shape)

    # Perform Singular Value Decomposition (SVD)
    U, s, Vt = randomized_svd(covariance_matrix, n_components=k, random_state=42)
    print("SVD Performed...")

    # Extract canonical correlation coefficients
    canonical_corr = np.sqrt(s[:min(rna_cca_data.shape[1], protein_cca_data.shape[1], k)])
    # print("Canonical Correlation Coefficients:", canonical_corr)

    # Extract canonical weights
    rna_weights = U[:rna_grouped.shape[1], :len(canonical_corr)]
    protein_weights = Vt.T[:protein_grouped.shape[1], :len(canonical_corr)]

    # Identify top RNA-Protein correlations
    top_pairs = []
    for i in range(len(canonical_corr)):
        rna_loading = np.abs(rna_weights[:, i])  # Magnitude of loadings for RNA
        protein_loading = np.abs(protein_weights[:, i])  # Magnitude of loadings for Protein

        # Pair each RNA with each Protein for this canonical component
        correlations = np.outer(rna_loading, protein_loading)

        # Find the indices of the top RNA-Protein pair for this component
        max_corr_index = np.unravel_index(np.argmax(correlations), correlations.shape)
        top_pairs.append((max_corr_index, correlations[max_corr_index]))

    # Extract and validate top RNA-Protein pairs
    correlated_pairs = []
    for i, (indices, corr_value) in enumerate(top_pairs):
        rna_idx, protein_idx = indices
        if rna_idx < len(rna_grouped.columns) and protein_idx < len(protein_grouped.columns):
            rna_feature = rna_grouped.columns[rna_idx]
            protein_feature = protein_grouped.columns[protein_idx]
            correlated_pairs.append([rna_feature, protein_feature])  # Save the pair

    correlated_pairs = correlated_pairs[:, ::-1]

    # Convert the list of pairs into a numpy array
    correlated_pairs_array = np.array(correlated_pairs, dtype=str)
    correlated_pairs_df = pd.DataFrame(correlated_pairs_array, columns=['Protein Name', 'RNA Name'])

    # Save the DataFrame to a CSV file
    correlated_pairs_df.to_csv(output_csv_path, index=False)
    print(f"Correlated pairs saved to {output_csv_path}")

    rna_protein_correspondence = []

    for i in range(correspondence.shape[0]):
        curr_protein_name, curr_rna_names = correspondence.iloc[i]
        if curr_protein_name not in protein_adata.var_names:
            continue
        if curr_rna_names.find('Ignore') != -1: # some correspondence ignored eg. protein isoform to one gene
            continue
        curr_rna_names = curr_rna_names.split('/') # eg. one protein to multiple genes
        for r in curr_rna_names:
            if r in rna_adata.var_names:
                rna_protein_correspondence.append([r, curr_protein_name])
    
    rna_protein_correspondence = np.array(rna_protein_correspondence)
    rna_protein_correspondence = np.concatenate((rna_protein_correspondence, correlated_pairs_array), axis=0)

    # Keep only the topmost (first) occurrence of each RNA and protein
    rna_to_protein = {}
    protein_to_rna = {}
    
    # Iterate forward
    for rna, protein in rna_protein_correspondence:
        if rna not in rna_to_protein and protein not in protein_to_rna:
            rna_to_protein[rna] = protein
            protein_to_rna[protein] = rna
    
    # Extract the unique pairs in the original order
    unique_pairs = np.array([[rna, rna_to_protein[rna]] for rna in rna_to_protein])
    
    rna_protein_correspondence = np.asarray(unique_pairs)
    rna_protein_correspondence = rna_protein_correspondence[:min(150, rna_protein_correspondence.shape[0])]
        
    # Return the resulting array
    return rna_protein_correspondence


In [11]:
protein = pd.read_csv("../data/citeseq_pbmc/pro.csv") # 10k cells (protein)
# convert to AnnData
protein_adata = ad.AnnData(
    protein.to_numpy(), dtype=np.float32
)
protein_adata.var_names = protein.columns

/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:401: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(


In [12]:
rna = mmread("../data/citeseq_pbmc/rna.txt") # rna count as sparse matrix, 10k cells (RNA)
rna_names = pd.read_csv('../data/citeseq_pbmc/citeseq_rna_names.csv')['names'].to_numpy()
# convert to AnnData
rna_adata = ad.AnnData(
    rna.tocsr(), dtype=np.float32
)
rna_adata.var_names = rna_names

/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:401: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(


In [13]:
metadata = pd.read_csv('../data/citeseq_pbmc/meta.csv')
labels_l1 = metadata['celltype.l1'].to_numpy()
labels_l2 = metadata['celltype.l2'].to_numpy()

protein_adata.obs['celltype.l1'] = labels_l1
protein_adata.obs['celltype.l2'] = labels_l2
rna_adata.obs['celltype.l1'] = labels_l1
rna_adata.obs['celltype.l2'] = labels_l2

In [ ]:
correspondence = pd.read_csv('protein_gene_conversion.csv')
correspondence.head()

,Protein name,RNA name
0,CD80,CD80
1,CD86,CD86
2,CD274,CD274
3,CD273,PDCD1LG2
4,CD275,ICOSLG


In [15]:
rna_protein_correspondence = compute_correlated_pairs(rna_adata, protein_adata, k=150,  correspondence = correspondence, output_csv_path='correlated_pairs.csv')

Data grouped...
Data Normalized...
Data Centered...
Covariance Matrix Computed...
(20953, 20953)
SVD Performed...
Correlated pairs saved to correlated_pairs.csv
